# Special Dunder and Magic methods

In this notebook you learn about special methods within our classes. Some people call this magic methods. Other call them dunder methods. It allows to emulate some builtin behavior within pyhton. And it is also how we implement operator overloading. 


An example of operator overloading is the print method. 


## `__str__` and `__repr__`

In [ ]:
print(1 + 2) #adding two integers together
print('a' + 'b') #adding two strings together

What we see that the behavior is different for adding two strings versus adding two integers. The behavior of the addition is different depending on the object (inter or string) This is done with the *overloading* principle. We can use this principle for our own objects as well. Normally when we print our object is gives a vague result. 

In [ ]:
class Bag:
    def __init__(self, items=list()):
        # An empty Bag can be represented by an empty list.
        self.inbag = items
        self.index = 0

my_bag = Bag()
print(my_bag)

It would be nice if we could change this behavior for something more informative. That is what a special method allow us to do. Special methods are surrounded by double underscores `__`. This is called **dunder** methods. Dunder here means “Double Under (Underscores). So far we used `__init__` method and the `__str__`. Another special method you should problably always implement next to `__init__` and `__str__` is `__repr__`

`__str__` is for a more readable representation when you print the object. `__repr__` is meant for debugging and logging, to be seen by other developpers. If an object does not have an `__str__` method is falls back to the `__repr__` method. So you should have that as a minimum

In [ ]:
class Bag:
    def __init__(self, items=list()):
        # An empty Bag can be represented by an empty list.
        self.inbag = items
        self.index = 0
 
    def __repr__(self):
        return "I'm a {} containing {} items".format(type(self).__name__, len(self.inbag))


my_bag = Bag()
print(my_bag)

In [10]:
class Bag:
    def __init__(self, items=list()):
        # An empty Bag can be represented by an empty list.
        self.inbag = items
        self.index = 0

    def __str__(self):
        # When a Bag is printed, a user-friendly message appears with the content of the Bag
        return "Your {} contains the following items: {}".format(type(self).__name__, self.inbag)
    
    def __repr__(self):
        return "I'm a {} containing {} items".format(type(self).__name__, len(self.inbag))


my_bag = Bag()
print(my_bag)

Your Bag contains the following items: []


I still can call the repr method as follow

In [13]:
print(repr(my_bag))

I'm a Bag containing 0 items


In [14]:
print(my_bag.__repr__())

I'm a Bag containing 0 items


---

## `__add__`

Remember the code  

In [1]:
print(1+2)
print("a"+"b")

3
ab


What actually happens in the background is that we call the integer `object.__add__` method. 

In [2]:
print(int.__add__(1,2))
print(str.__add__("a","b"))

3
ab


In [5]:
help(str)

Help on class str in module builtins:

class str(object)
 |  str(object='') -> str
 |  str(bytes_or_buffer[, encoding[, errors]]) -> str
 |  
 |  Create a new string object from the given object. If encoding or
 |  errors is specified, then the object must expose a data buffer
 |  that will be decoded using the given encoding and error handler.
 |  Otherwise, returns the result of object.__str__() (if defined)
 |  or repr(object).
 |  encoding defaults to sys.getdefaultencoding().
 |  errors defaults to 'strict'.
 |  
 |  Methods defined here:
 |  
 |  __add__(self, value, /)
 |      Return self+value.
 |  
 |  __contains__(self, key, /)
 |      Return key in self.
 |  
 |  __eq__(self, value, /)
 |      Return self==value.
 |  
 |  __format__(...)
 |      S.__format__(format_spec) -> str
 |      
 |      Return a formatted version of S as described by format_spec.
 |  
 |  __ge__(self, value, /)
 |      Return self>=value.
 |  
 |  __getattribute__(self, name, /)
 |      Return getatt

So for each object there is an `__add__` method with it's own implementation. This means we can overide the `__add__` method with our own implementation in our own objects

In [8]:
class Bag:
    def __init__(self, items=list()):
        # An empty Bag can be represented by an empty list.
        self.inbag = items
        self.index = 0

    def __str__(self):
        # When a Bag is printed, a user-friendly message appears with the content of the Bag
        return "Your {} contains the following items: {}".format(type(self).__name__, self.inbag)
    
    def __repr__(self):
        return "I'm a {} containing {} items".format(type(self).__name__, len(self.inbag))


    def __add__(self, other):
        # When two bags are added to each other (+) all items in both Bags will be placed in a new Bag
        return Bag(self.inbag + other.inbag)

little_bag = Bag(["Milk"])
big_bag = Bag(["Ananas", "Patatoes"])
little_bag = little_bag + big_bag
print(little_bag)

Your Bag contains the following items: ['Milk', 'Ananas', 'Patatoes']


---

## `__iter__`

Whenever the interpeter needs to iterate over an object, it automatically calls `iter()`. The iter built-in function
1. checks whether the object implements `__iter__`, and calls that to obtain an interator
2. if `__iter__` is not implemented but `__getitem__` is implemented, Python creates an iterator that attemps to fetch items in order, starting from index 0
3. if that fails, Python raises TypeError, usually saying "object is not iterable". 

Any object that implement an `__iter__` method should return an object that is iterable. Sequences are iterable; as are objects implementing an `__getitem__` method that takes 0 based indexes. Python obtain iterators from iterables. But there is a difference between an iterable and an iterator. An iterable is an object you can iterate over. An iterator is an activity of itteration. An object can next to being suitable for itteration (implementation of `__iter__`) also implement the itteration activity. We do such by implementing `__next__`. So the standard interface of the object is 

    __iter__
       returns self; this allows iterators to be used where an iterable is expected, for example a loop
     
    __next__
       returns the next available item, raising StopIteration when there are no more items

In [15]:
class Bag:
    def __init__(self, items=list()):
        # An empty Bag can be represented by an empty list.
        self.inbag = items
        self.index = 0

    def __iter__(self):
        # The items in a list can be traversed using a for loop and Python provides for user-defined iterators that
        # be used with a Bag.
        return self

    def __next__(self):
        if self.index == len(self.inbag):
            self.index = 0
            raise StopIteration
        else:
            self.index += 1
            return self.inbag[self.index-1]

In [16]:
lovely_bag = Bag(["Cola", "Chips"])
print(next(lovely_bag))
print(next(lovely_bag))
print(next(lovely_bag))


Cola
Chips


StopIteration: 

An even nicer implementation of the `__iter__` is the usage of a generator function

In [ ]:
class Bag:
    def __init__(self, items=list()):
        # An empty Bag can be represented by an empty list.
        self.inbag = items
        self.index = 0

    def __iter__(self):
        for item in self.inbag:
            yield item
            
    def __next__(self):
        if self.index == len(self.inbag):
            self.index = 0
            raise StopIteration
        else:
            self.index += 1
            return self.inbag[self.index-1]


In [ ]:
lovely_bag = Bag(["Cola", "Chips"])
print(*iter(lovely_bag))
print(next(lovely_bag))
print(next(lovely_bag))

---

## Assignment 1

Review the following code and answer the following

1. Which python built in methods are overloaded?
2. How many dunder methods do you count?
3. What is the difference between the dunder methods and the regular methods?


In [17]:
class Bag:
    def __init__(self, items=list()):
        # An empty Bag can be represented by an empty list.
        self.inbag = items
        self.index = 0

    def __len__(self):
        # The size of the Bag can be determined by the size of the list
        return len(self.inbag)

    def __contains__(self, item):
        # Determining if the Bag contains a specific item can be done using the equivalent list operation
        return item in self.inbag

    def __bool__(self):
        # Boolean test can be performed on the Bag
        return self.inbag

    def __str__(self):
        # When a Bag is printed, a user-friendly message appears with the content of the Bag
        return "Your {} contains the following items: {}".format(type(self).__name__, self.inbag)

    def append(self, item):
        # When a new item is added to the Bag, it can be appended to the end of the list since there is no specific
        # ordering of the items in a bag.
        self.inbag.append(item)

    def remove(self, item):
        # Removing an item from the Bag can also be handled by the equivalent list operation
        self.inbag.remove(item)

    def __iter__(self):
        # The items in a list can be traversed using a for loop and Python provides for user-defined iterators that
        # be used with a Bag.
        return self

    def __next__(self):
        if self.index == len(self.inbag):
            self.index = 0
            raise StopIteration
        else:
            self.index += 1
            return self.inbag[self.index-1]

    def __eq__(self, other):
        # Multiple Bags can be sorted/compared based on the number of items in the Bag
        return len(self.inbag) == len(other.inbag)

    def __gt__(self, other):
        return len(self.inbag) > len(other.inbag)

    def __lt__(self, other):
        return len(self.inbag) < len(other.inbag)

    def __repr__(self):
        return "I'm a {} containing {} items".format(type(self).__name__, len(self.inbag))

    def __add__(self, other):
        # When two bags are added to each other (+) all items in both Bags will be placed in a new Bag
        return Bag(self.inbag + other.inbag)


## Assignment 2

Write a test case for each dunder method. Call the builtin method to test the dunder method. For instance to test the `__len__` method write code such as below:

In [18]:
boodschappenlijst = ["Chocolate", "Cake"]
bag = Bag(boodschappenlijst)
len(boodschappenlijst) == len(bag)




True

## Assignment 3

Find on the internet a webpage that describes the working of the dunder method `__call__`. Explain in your own words how it works

In [ ]:
dir(int)

See https://rszalski.github.io/magicmethods/